## Import Libraries  & Utils

In [8]:
import sys
sys.path.append('/paths_generator.py')
import paths_generator as pg
import random as rand
import math as m
import numpy as np
from scipy.optimize import dual_annealing
import pdb

zones_num = pg.region.shape[0]*pg.region.shape[1]
print(zones_num)

81


## Opti problem

#### Constraint Function

In [3]:
# It works just for one asset
def constraint(x): 
    
    constraints = []

    for asset in pg.assets:
      for road in pg.T_Ci[asset]:
        temp = []
        
        for i in range(len(road)-1):
          temp.append(x[road[i+1]][road[i]])

        constraints.append(1-sum(temp))   # All of them should be < 0
    
    return np.array(constraints)

#### Objective function (unconstrained problem)

In [4]:
# Define the objective function
def objective(x, penalty=1e5): 
    print("dkhelt", x)
    S = 0
    x = np.array(x)
    x = np.reshape(x,(pg.zones_number, pg.zones_number))
    for i in range(zones_num):
        e_i = pg.region[pg.Rcount(i)] 
        for j in range(zones_num):
            S += x[i][j] * max(0, pg.slr - e_i)
    # Apply the constraint function and add penalties for violations
    constraints = constraint(x)
    if np.any(constraints > 0):
        S += penalty * np.sum(constraints)**2   # On applique le lagrangien!  # I don't need le min(max): it's defined explicitly
    
    print("9rib n returni!")
    return S 

## Dual Annealing

In [5]:
# Define the bounds for the decision variables
bounds = [(0, 1) for i in range(zones_num) for j in range(zones_num)]

# Call the Dual Annealing optimizer
result = dual_annealing(func=objective, bounds=bounds, maxiter=1000, seed=123, int = True)

# Print the optimal solution
print(result.x)
print(result.fun)

TypeError: dual_annealing() got an unexpected keyword argument 'int'

In [ ]:
breakpoint()